In [1]:
# CS 6316 ML Project: LASSO
# Created on Wed Apr 29 22:23 2020 
# @author: sl4bq

import time
import os
import shutil
import pandas as pd
from datetime import datetime
from dateutil import tz
import warnings
import itertools

import math
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split



In [2]:
# Remove the NaN rows and columns

def removeNaN(curr_feat_1, curr_feat_2):
    curr_feat_1_nan = np.array([])  # numpy.float64
    curr_feat_2_nan = np.array([])  # numpy.float64
    posi = 0
    # *** Check for and remove the Nan rows in from ONLY [curr_feat_2] feature rows
    for posi in range(len(curr_feat_2)):
        if not (math.isnan(curr_feat_2[posi])):# and not(math.isnan(curr_feat_1[posi]))):
            curr_feat_1_nan = np.append(curr_feat_1_nan, curr_feat_1[posi])
            curr_feat_2_nan = np.append(curr_feat_2_nan, curr_feat_2[posi])
            if (posi < 10):
                print(curr_feat_2_nan[posi])
        if (math.isnan(curr_feat_1[posi])):
            # curr_feat_1_nan = np.append(curr_feat_1_nan, curr_feat_1[posi])
            # curr_feat_2_nan = np.append(curr_feat_2_nan, curr_feat_2[posi])
            if (posi < 10):
                print(curr_feat_2_nan[posi])
    return curr_feat_1_nan, curr_feat_2_nan

#### Import .csv dataset into pandas
print('Importing Data Now')

#filename = '/Users/Mandala/PycharmProjects/ML_Project/windowing_results.csv'
filename = 'actrec_windowing_3&4.csv'

# Read data from csv file 
df_feat = pd.read_csv(filename)
df_keys = df_feat.keys()



Importing Data Now


In [3]:
df_feat

,Unnamed: 0,ECG #,start_time,end_time,acc_mean,acc_mean.1,acc_50%,acc_max,acc_eng,acc_var,...,gyro_mean.1,gyro_50%,gyro_max,gyro_eng,gyro_var,gyro_skew,gyro_kurt,gyro_entropy,gyro_mad,Activity
0,svm,3,2019-11-21 09:23:04,2019-11-21 09:23:06,3.074808,3.045923,3.078563,3.116174,21537.502717,0.000125,...,0.125965,1.410155,9.516461,12581.802896,2.238610,2.099189,5.761027,7.729836,1.063948,"(0,0)"
1,svm,3,2019-11-21 09:23:05,2019-11-21 09:23:07,3.070734,3.045923,3.070070,3.109342,21480.392958,0.000086,...,0.047682,1.113528,9.276601,9444.200326,1.857098,2.559431,8.533665,7.729227,0.922118,"(0,0)"
2,svm,3,2019-11-21 09:23:06,2019-11-21 09:23:08,3.070877,3.045923,3.069935,3.100552,21454.066753,0.000077,...,0.037221,1.028702,9.276601,9565.669730,2.037144,2.691735,8.812490,7.727298,0.940215,"(0,0)"
3,svm,3,2019-11-21 09:23:07,2019-11-21 09:23:09,3.072879,3.054522,3.070070,3.102285,21463.128781,0.000059,...,0.037221,1.415488,9.276601,17090.836235,3.175637,1.601641,2.546772,7.727636,1.370081,"(0,0)"
4,svm,3,2019-11-21 09:23:08,2019-11-21 09:23:10,3.073298,3.054522,3.069935,3.102285,21478.429820,0.000060,...,0.037221,1.443367,8.908689,16416.036159,2.970355,1.593336,2.500608,7.726857,1.327227,"(0,0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3901,svm,4,2019-11-21 10:19:30,2019-11-21 10:19:32,11.702805,11.417292,11.703640,11.895382,262961.324078,0.003379,...,0.167190,1.762409,5.488829,9116.831689,0.861082,0.795411,0.229283,7.560080,0.752801,"(2,3)"
3902,svm,4,2019-11-21 10:19:31,2019-11-21 10:19:33,11.695273,11.417292,11.694066,11.895382,263581.173677,0.003774,...,0.163304,1.957578,5.488829,9871.091583,0.889702,0.602416,0.097293,7.563720,0.757811,"(2,3)"
3903,svm,4,2019-11-21 10:19:32,2019-11-21 10:19:34,11.692662,11.417292,11.694293,11.872994,265650.063209,0.003243,...,0.123713,1.751808,5.570260,8338.294151,0.862812,1.014820,1.410757,7.571988,0.706481,"(2,3)"
3904,svm,4,2019-11-21 10:19:33,2019-11-21 10:19:35,11.703943,11.423132,11.703998,11.872994,266023.905128,0.002225,...,0.123713,1.529116,5.570260,6835.576335,0.737167,1.142133,1.965215,7.571474,0.654387,"(2,3)"


In [4]:
print(df_keys)
len(df_keys)

Index(['Unnamed: 0', 'ECG #', 'start_time', 'end_time', 'acc_mean',
       'acc_mean.1', 'acc_50%', 'acc_max', 'acc_eng', 'acc_var', 'acc_skew',
       'acc_kurt', 'acc_entropy', 'acc_mad', 'gyro_mean', 'gyro_mean.1',
       'gyro_50%', 'gyro_max', 'gyro_eng', 'gyro_var', 'gyro_skew',
       'gyro_kurt', 'gyro_entropy', 'gyro_mad', 'Activity'],
      dtype='object')


25

In [5]:
# Data Preprocessing
df = df_feat.copy()
df = df.dropna()

Xs = df[df_keys[4:24]]
y = df['Activity']

print(Xs)
print(y)

       acc_mean  acc_mean.1    acc_50%    acc_max        acc_eng   acc_var  \
0      3.074808    3.045923   3.078563   3.116174   21537.502717  0.000125   
1      3.070734    3.045923   3.070070   3.109342   21480.392958  0.000086   
2      3.070877    3.045923   3.069935   3.100552   21454.066753  0.000077   
3      3.072879    3.054522   3.070070   3.102285   21463.128781  0.000059   
4      3.073298    3.054522   3.069935   3.102285   21478.429820  0.000060   
...         ...         ...        ...        ...            ...       ...   
3901  11.702805   11.417292  11.703640  11.895382  262961.324078  0.003379   
3902  11.695273   11.417292  11.694066  11.895382  263581.173677  0.003774   
3903  11.692662   11.417292  11.694293  11.872994  265650.063209  0.003243   
3904  11.703943   11.423132  11.703998  11.872994  266023.905128  0.002225   
3905  11.709427   11.502270  11.708944  11.872994  177560.567817  0.001717   

      acc_skew  acc_kurt  acc_entropy   acc_mad  gyro_mean  gyr

In [6]:
# This is the data split

X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2, random_state=1)


In [7]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
from sklearn.ensemble import RandomForestClassifier

m = RandomForestClassifier(n_estimators=20, random_state=0)
m.fit(X_train, y_train)
y_pred = m.predict(X_test)

In [9]:
from sklearn.metrics import confusion_matrix, accuracy_score

print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[ 33   0   0   0   0   2   2   0   0   0   1   0]
 [  2  38   6   0   0   0   2   0   0   0   1   0]
 [  2   6  84   0   0   2   5   0   0   1   0   0]
 [  0   0   2  67   1   2   5   0   0   0   2   1]
 [  0   0   0   0  11   1   0   0   0   0   0   0]
 [  0   0   1   2   0  61   1   0   0   0   0   0]
 [  0   0   0   3   0   0 121   1   0   1   3   1]
 [  1   0   0   0   0   1   1  55   0   1   2   1]
 [  0   0   0   0   0   0   1   0  10   0   1   0]
 [  0   0   4   2   0   0   1   2   0  57   0   2]
 [  0   4   0   0   0   0   4   0   1   0  60   1]
 [  1   0   0   2   0   0   2   0   0   1   0  90]]
0.8785166240409207


In [10]:
df_video = df.loc[(df.Activity == "(0,0)") | (df.Activity == "(0,1)") | (df.Activity == "(0,2)") | (df.Activity == "(0,3)")]
df_noeval = df.loc[(df.Activity == "(1,0)") | (df.Activity == "(1,1)") | (df.Activity == "(1,2)") | (df.Activity == "(1,3)")]
df_eval = df.loc[(df.Activity == "(2,0)") | (df.Activity == "(2,1)") | (df.Activity == "(2,2)") | (df.Activity == "(2,3)")]

In [11]:
def RFC(x):
    z = x.copy()
    df = z.dropna()

    Xs = df[df_keys[4:18]]
    y = df['Activity']
    
    X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2, random_state=10)
    
    m = RandomForestClassifier(n_estimators=20, random_state=0)
    m.fit(X_train, y_train)

    y_pred = m.predict(X_test)

    print(confusion_matrix(y_test, y_pred))
    print(accuracy_score(y_test, y_pred))

In [12]:
RFC(df_video)

[[33  1  3  0]
 [ 0 42  7  3]
 [ 2  5 81  3]
 [ 3  0  4 66]]
0.8774703557312253


In [13]:
RFC(df_noeval)

[[ 13   1   0   0]
 [  2  51   4   0]
 [  1   1 139   1]
 [  0   0   1  56]]
0.9592592592592593


In [14]:
RFC(df_eval)

[[13  3  2  0]
 [ 0 73  3  0]
 [ 0  1 77  4]
 [ 0  5  2 76]]
0.9227799227799228
